# C++ Jezgro


Po pokretanju vec je automatski izvrsena celija:
    #include <iostream>
    using namespace std;
    
%r je line magic i izvrsava kod kao sto bi bio izvrsen u main funkciji

%%r je cell magic i celu celiju tretira kao da je main funkcija

In [20]:
int a = 2;
%r cout << a << endl;

string b  = "Hello";
%r cout << b << endl;

// samo deklarisemo funkciju
void f(int i, const string & s){
    while(i--)
        cout << i << ": " << s << endl;
}



2
Hello


In [21]:
// deklarisemo jos jednu funkciju
void g(void){
    f(a, b);
}

In [22]:
%%r 
g();                                                                     
a = 5;                                                                       
f(a, "World");  

4: Hello
3: Hello
2: Hello
1: Hello
0: Hello
4: World
3: World
2: World
1: World
0: World


# Problem je redosled pretrazivanja za linkovanje

In [4]:
long a = 3333;
%r cout << a << endl;

3333


Uveli smo novu definiciju za a (long) i to izgleda kao da radi medjutim. 

AKo ponovo pokusamo da pristupimo 'a' u drugoj celiji linker ce naci ono prov int a... :(

In [5]:
%%r 
cout << a; // nasao je staro a :(
a = 42; // ali 

3

Ali mozemo da menjamo vrednosti bez problema

sto nam ne pomaze bas za funkcije (jedino da uvek koristimo)
pokazivace  na funkcije, 

onda bi mogli da im menjamo definiciju f-je, promenom vrednosti pokazivaca.

In [9]:
%r cout << a;

42

## Primeri greske kompilacije i linkinga

In [11]:
int ;

error: declaration does not declare anything [-fpermissive]
 int ;
 ^


In [12]:
extern int c;

%r a = c;

undefined symbol: c

### Segmentation fault nazalost ubija jezgro :( ...